Loosely following this tutorial on huggingface here : [Open LLMs Using TGI’s Messages API](https://huggingface.co/learn/cookbook/tgi_messages_api_demo)

In [1]:
!pip install --upgrade -q langchain langchain-community langchainhub langchain-openai llama-index


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
!pip install --upgrade -q gradio


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
BASE_URL = 'https://api.upstage.ai/v1/solar'
APIKEY = ''

In [2]:
from openai import OpenAI
import os

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat_model = ChatOpenAI(base_url=BASE_URL, model_name="solar-1-mini-chat", api_key=APIKEY)
messages = [
    SystemMessage(
        content="You are a helpful assistant."
    ),
    HumanMessage(
        content="Hi, how are you?"
    )
]

response = chat_model.invoke(messages)
print(response)

content='Hello! As an artificial intelligence, I do not have feelings or emotions. However, I am here to help you with any questions or tasks you may have. How can I assist you today?'


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=APIKEY,
    base_url=BASE_URL
)

# this system prompt will be used globally on the chat session.
system_prompt = {
    "role": "system",
    "content": "You are a helpful assistant."
}

model = "solar-1-mini-chat"

# Keep chat history in a list.
chat_history = []

# Set a limit for the chat history to manage the token count.
history_size = 10

while True:
    # Step 1: Get user input.
    user_prompt = input("User: ")
    chat_history.append({
        "role": "user",
        "content": user_prompt
    })

    # Step 2: Use the chat history to generate the chatbot's response.
    messages = [system_prompt] + chat_history
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )

    # Step 3: Output the Solar response chunks.
    print("SOLAR: ", end="")
    solar_response = ""
    for chunk in stream:
        response_content = chunk.choices[0].delta.content
        if response_content is not None:
            solar_response += response_content
            print(response_content, end="")
    print() # Finish response output.

    # Step 4: Append the Solar response to the chat history.
    chat_history.append({
        "role": "assistant",
        "content": solar_response
    })

    # Step 5: Ensure the chat history doesn't exceed the size limit.
    chat_history = chat_history[:history_size]


In [9]:
import gradio as gr
import random
import time

client = OpenAI(
    api_key=APIKEY,
    base_url=BASE_URL
)

# this system prompt will be used globally on the chat session.
system_prompt = {
    "role": "system",
    "content": "You are a helpful assistant."
}

model = "solar-1-mini-chat"

# Keep chat history in a list.
chat_history = []

# Set a limit for the chat history to manage the token count.
history_size = 10

def doResponse(user_prompt):
    # Step 1: Get user input.
    chat_history.append({
        "role": "user",
        "content": user_prompt
    })

    # Step 2: Use the chat history to generate the chatbot's response.
    messages = [system_prompt] + chat_history
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )

    # Step 3: Output the Solar response chunks.
    solar_response = ""
    for chunk in stream:
        response_content = chunk.choices[0].delta.content
        if response_content is not None:
            solar_response += response_content
            print(response_content, end="")
    print() # Finish response output.
    
    # Step 4: Append the Solar response to the chat history.
    chat_history.append({
        "role": "assistant",
        "content": solar_response
    })

    # Step 5: Ensure the chat history doesn't exceed the size limit.
    chat_history = chat_history[:history_size]
    return chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        time.sleep(2)
        return "", doResponse(message)

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
